In [9]:
from collections import defaultdict, Counter
from copy import copy
from math import exp, sqrt
from random import random, shuffle, choice, randint, uniform

from keras import Input, Model
from keras.callbacks import EarlyStopping
from keras.constraints import non_neg, max_norm
import numpy
from numpy import array, mean, ones
from pandas import concat
from pandas import DataFrame
from keras.models import Sequential
from keras.layers import LSTM, multiply, subtract, add, Activation, Lambda, division
from keras.layers import Dense, concatenate, MaxPooling1D, LocallyConnected1D, Reshape
from keras import backend as K
from keras import constraints
from keras.optimizers import Adam, RMSprop, SGD

from utils import generate_student_name
import random

nq = 12
n_traits = 1
tr_len = 100
tt_len = 10


In [2]:
generate_student_name()

'YYFO NOJU '

In [3]:
from keras import backend as K
from keras.constraints import Constraint
from keras.engine.topology import Layer
from keras import initializers

class WeightClip(Constraint):
    '''Clips the weights incident to each hidden unit to be inside a range
    '''
    def __init__(self, min_w=0, max_w=4):
        self.min_w = min_w
        self.max_w = max_w

    def __call__(self, p):
        return K.clip(p, self.min_w, self.max_w)

    def get_config(self):
        return {'name': self.__class__.__name__,
                'min_w': self.min_w,
                'max_w': self.max_w }


class ProductLayer(Layer):

    def __init__(self, output_dim, kernel_constraint=WeightClip(min_w=0.0, max_w=4.0), **kwargs):
        self.output_dim = output_dim
        super(ProductLayer, self).__init__(**kwargs)
        self.kernel_constraint = constraints.get(kernel_constraint)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(1, self.output_dim),
                                      initializer=initializers.RandomUniform(minval=0.0, maxval=4.0), #.Constant(value=2.0),
                                      trainable=True,
                                      constraint=self.kernel_constraint)
        
        super(ProductLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
#         print("sk",self.kernel.shape)
#         print("x", x.shape)
#         self.kernel = K.clip(self.kernel, self.cmin, self.cmax)
        z = K.concatenate([x,self.kernel], axis=0)
#         print("internal z shape:", z.shape)
        p = K.prod(z, axis=0)
#         p = K.clip(p, self.cmin, self.cmax)
#         self.kernel = K.clip(self.kernel, self.cmin, self.cmax)

#         print("internal output shape:", p.shape)
        return p

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

In [4]:
class Question():
    def __init__(self, qix, nt=10, optimiser=None):
        self.id = qix
        #self.qvals = [ int(uniform(0,1)>=0.5) for _ in range(nt) ]
        self.beta = randint(0,4)
        print("beta=",self.beta)
        
        th = Input(shape=(1,), name="user_theta")
        b_in = Input(shape=(1,), name="beta_in")
        bvar = ProductLayer(1) # introduce a trainable weight
        b = bvar(b_in)
        dif = subtract([th,b])
#         Pr = Dense(1, activation="sigmoid")(dif)
        Pr = Lambda(lambda z: 1.0 / (1.0 + K.exp(-z)), name="Pr_sigmoid")(dif)
        model = Model(inputs=[th,b_in], outputs=[Pr])
        #o = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
        o = optimiser
        model.compile(optimizer=o, loss="binary_crossentropy")
        self.model = model
        self.pred_beta = bvar
        self.pred_beta_sc = 2.0
        
# qs = [Question(qix, n_traits) for qix in range(10)]
# qs[0].model.summary()

In [5]:
def generate_student_struct(nt=10, nq=20, optimiser=None):
    name = generate_student_name()
    theta = randint(0,4)
    mastery = [0 for _ in range(nq)]
    
    dict = {}
    dict["name"] = name
    dict["theta"] = theta
#     dict["h_practice"] = h_practice
#     dict["o_practice"] = o_practice
#     dict["mastery"] = mastery
    
    th_in = Input(shape=(1,), name="user_theta")
    b = Input(shape=(1,), name="beta_in")
    th_layer = ProductLayer(1) # introduce a trainable weight
    th = th_layer(th_in)
    dif = subtract([th,b])
    #Pr = Dense(1, activation="sigmoid")(dif)
    Pr = Lambda(lambda z: 1.0 / (1.0 + K.exp(-z)), name="Pr_sigmoid")(dif)
    model = Model(inputs=[th_in,b], outputs=[Pr])
    #o = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
    o = optimiser
    model.compile(optimizer=o, loss="binary_crossentropy")
    dict["model"] = model
    dict["pred_theta"] = th_layer
    dict["pred_theta_sc"] = 2.0
    
#     dict["q_network"] = create_q_network(nq)
#     dict["q_passes"] = [ 0 for _ in range(nq) ]
    
    return dict

In [6]:
def attempt_q(student, q: Question, update_weights=True):
    real_theta = student["theta"]
    real_beta = q.beta
    p = calculate_pass_probability(real_theta, real_beta)

    this_att = uniform(0,1)
    if (this_att <= p):
        passed=1
        student["mastery"][q.id] = 1
    else:
        passed=0

#     if update_weights:
#         pred_beta = q.pred_beta.get_weights()
# #     print(pred_beta)
#         pred_theta = student["pred_theta"].get_weights()
#         print("true th:", student["theta"])
#         print("true beta:", q.beta)
        
#         print("pre ptheta:", pred_theta)
#         print("pre pbeta:", pred_beta)
#         obi = numpy.ones(1)
#         print(obi)
#         es = EarlyStopping(monitor="loss", mode="auto")
#         q.model.fit([pred_theta[0][0], obi],[array([passed])], batch_size=1, verbose=0, epochs=10, callbacks=[es])
#         student["model"].fit([obi, pred_beta[0][0]],[array([passed])], batch_size=1, verbose=0, epochs=1, callbacks=[es])
#         print("post ptheta:", student["pred_theta"].get_weights()[0][0])
#         print("post pbeta:", q.pred_beta.get_weights()[0][0])
        
    return passed

In [7]:
def calculate_pass_probability(a, beta):
    z = (a - beta)
    print("z={}".format(z))
    try:
        p_pass = 1.0 / (1.0 + exp(-z))
        print("p_pass={}".format(p_pass))
    except OverflowError:
        p_pass = 0.0
    #print("real p_pass = {}".format(p_pass))
    return p_pass
    

In [10]:
random.seed(666)
opt = Adam(lr=0.1)
master_qs = [Question(qix, n_traits, optimiser=opt) for qix in range(nq)]
for q in master_qs:
    print("Q:{}, difficulty={:.2f} across 1 components".format(q.id, q.beta))

beta= 3
beta= 3
beta= 3
beta= 2
beta= 4
beta= 0
beta= 4
beta= 4
beta= 2
beta= 0
beta= 0
beta= 1
Q:0, difficulty=3.00 across 1 components
Q:1, difficulty=3.00 across 1 components
Q:2, difficulty=3.00 across 1 components
Q:3, difficulty=2.00 across 1 components
Q:4, difficulty=4.00 across 1 components
Q:5, difficulty=0.00 across 1 components
Q:6, difficulty=4.00 across 1 components
Q:7, difficulty=4.00 across 1 components
Q:8, difficulty=2.00 across 1 components
Q:9, difficulty=0.00 across 1 components
Q:10, difficulty=0.00 across 1 components
Q:11, difficulty=1.00 across 1 components


# Training
This is where sh!t gets real.  We take our tr_len (1000?) students, and iterate over them 100 times to create 100,000 *complete examples* of a student attacking the curriculum.  The questions themselves are attacked in random order: the student has no intelligent guidance throught the material. (Obvious we may wish to provide that guidance at some point in the future.)

Remember, there are only 12 exercises in the curriculum, so if the student is taking 60 or 70 goes to answer them all, that's pretty poor.  But some of these kids are dumb as lumber, so cut them some slack!  They will all get there in the end since by the CMU AFM practice will, eventually, make perfect!

In [9]:
MIN_THETA =0
MAX_THETA =4
def calc_better_theta(curr_theta, attempts):
    eps = 0.1
    sum_spread=0.0
    sum_info = 0.0
    for ix, (psi,q_i,pf) in enumerate(attempts):
        #go through the answered questions, sum up the difference in value vs expected value
        u_i = float( pf ) #cast bool of response to float i.e. {1,0}
        p_i_theta = calculate_pass_probability(curr_theta, q_i.pred_beta_sc)
        print("real/exp",u_i,p_i_theta)
        sum_spread += ( u_i - p_i_theta ) #TODO check this -ve sign
        #Then get the information function, tells us about precision of estimate
        info = p_i_theta * (1.0-p_i_theta)
        sum_info += info
    print("sum_spread=",sum_spread, "scaled_sp=",(sum_spread/sum_info))
    print("total info", sum_info)
    new_theta = curr_theta + (sum_spread / sum_info)
    print("raw new theta=", new_theta)

    if new_theta >= MAX_THETA:
        new_theta = MAX_THETA
        return new_theta
    elif new_theta <= MIN_THETA:
        new_theta = MIN_THETA
        return new_theta
    elif abs(new_theta - curr_theta) < eps:
        return new_theta # we're done, new theta is calc'd
    else:
        return calc_better_theta(new_theta, attempts) # iterate again

def calc_better_beta(curr_beta, attempts):
    eps = 0.1
    sum_spread=0.0
    sum_info = 0.0
    psis, qs, pfs = zip(*attempts)
    for ix,psi in enumerate(psis):
        th = psi["pred_theta_sc"]
        #go through the answered questions, sum up the difference in value vs expected value
        u_i = float( pfs[ix] ) #cast bool of response to float i.e. {1,0}
        p_i = calculate_pass_probability(th, curr_beta)
        print("Q real/exp",u_i,p_i)
        sum_spread += ( u_i - p_i ) #TODO check this -ve sign
        #Then get the information function, tells us about precision of estimate
        info = p_i * (1.0-p_i)
        sum_info += info
    print("Q sum_spread=",sum_spread, "scaled_sp=",(sum_spread/sum_info))
    print("Q total info", sum_info)
    new_beta = curr_beta - (sum_spread / sum_info)
    print("Q raw new beta=", new_beta)

    if new_beta >= MAX_THETA:
        new_beta = MAX_THETA
        return new_beta
    elif new_beta <= MIN_THETA:
        new_beta = MIN_THETA
        return new_beta
    elif abs(new_beta - curr_beta) < eps:
        return new_beta # we're done, new theta is calc'd
    else:
        return calc_better_beta(new_beta, attempts) # iterate again
    


In [11]:
Xs = {}
Tz = {}
ys = {}
alphas = {}
attempts = []
attempts_by_psi = {}
attempts_by_q = {}
no_tr_students = 100

random.seed(666)
psi_list = [ generate_student_struct(optimiser=opt) for _ in range(no_tr_students)]

user_budget = 10
for run in range(1):
    print("----{}\n".format(run))
    for psi in psi_list:
        ct = 0
        psi["mastery"] = [0 for _ in range(nq)]
        qs = [ix for ix in range(len(master_qs))]
        print("* * * **** USER {}".format(psi["name"]))
        print("* * * * ** THETA {}".format(psi["theta"]))
            
        while(True):
            qix = random.choice(qs)
            q = master_qs[qix]
            passed=False
#             while not passed:
#             print(qix, q.pred_beta)
#             if qix not in Xs:
#                 Xs[qix]=[]
#                 Tz[qix]=[]
#                 ys[qix]=[]
#                 alphas[qix]=[]
            if psi["name"] not in attempts_by_psi:
                attempts_by_psi[psi["name"]]=[]
            
            if q not in attempts_by_q:
                attempts_by_q[q]=[]
    
            #print("attempting {}".format(q.id))
            passed = attempt_q(psi, q, update_weights=False)
            
            tup = (psi, q, passed)
            attempts.append(tup)
            attempts_by_psi[psi["name"]].append(tup)
            attempts_by_q[q].append(tup)
                        
            ct+=1
#             if passed:
                #print("passed")
            qs.remove(qix)

            if qs == []: # or ct>=user_budget:
                print("* ** *QFIN USER {}".format(psi["name"]))
                break

----0

* * * **** USER PHUTHY CORE 
* * * * ** THETA 0
z=-4
p_pass=0.01798620996209156
z=-4
p_pass=0.01798620996209156
z=-3
p_pass=0.04742587317756678
z=-3
p_pass=0.04742587317756678
z=-2
p_pass=0.11920292202211755
z=-4
p_pass=0.01798620996209156
z=0
p_pass=0.5
z=-3
p_pass=0.04742587317756678
z=0
p_pass=0.5
z=0
p_pass=0.5
z=-1
p_pass=0.2689414213699951
z=-2
p_pass=0.11920292202211755
* ** *QFIN USER PHUTHY CORE 
* * * **** USER DYG WOLA 
* * * * ** THETA 2
z=0
p_pass=0.5
z=0
p_pass=0.5
z=-1
p_pass=0.2689414213699951
z=-2
p_pass=0.11920292202211755
z=2
p_pass=0.8807970779778823
z=1
p_pass=0.7310585786300049
z=-1
p_pass=0.2689414213699951
z=2
p_pass=0.8807970779778823
z=-2
p_pass=0.11920292202211755
z=2
p_pass=0.8807970779778823
z=-2
p_pass=0.11920292202211755
z=-1
p_pass=0.2689414213699951
* ** *QFIN USER DYG WOLA 
* * * **** USER CYT BAG 
* * * * ** THETA 2
z=2
p_pass=0.8807970779778823
z=-2
p_pass=0.11920292202211755
z=2
p_pass=0.8807970779778823
z=2
p_pass=0.8807970779778823
z=0
p_pa

z=2
p_pass=0.8807970779778823
z=-1
p_pass=0.2689414213699951
z=-2
p_pass=0.11920292202211755
z=-1
p_pass=0.2689414213699951
z=-1
p_pass=0.2689414213699951
z=-2
p_pass=0.11920292202211755
z=2
p_pass=0.8807970779778823
z=-2
p_pass=0.11920292202211755
* ** *QFIN USER SIN YEPHI 
* * * **** USER POCOG THEM 
* * * * ** THETA 1
z=1
p_pass=0.7310585786300049
z=-1
p_pass=0.2689414213699951
z=-3
p_pass=0.04742587317756678
z=0
p_pass=0.5
z=1
p_pass=0.7310585786300049
z=-2
p_pass=0.11920292202211755
z=-3
p_pass=0.04742587317756678
z=-1
p_pass=0.2689414213699951
z=-2
p_pass=0.11920292202211755
z=-3
p_pass=0.04742587317756678
z=1
p_pass=0.7310585786300049
z=-2
p_pass=0.11920292202211755
* ** *QFIN USER POCOG THEM 
* * * **** USER RUP JANYT 
* * * * ** THETA 4
z=4
p_pass=0.9820137900379085
z=4
p_pass=0.9820137900379085
z=0
p_pass=0.5
z=4
p_pass=0.9820137900379085
z=2
p_pass=0.8807970779778823
z=1
p_pass=0.7310585786300049
z=1
p_pass=0.7310585786300049
z=0
p_pass=0.5
z=3
p_pass=0.9525741268224334
z=2


In [13]:
obi = numpy.ones(1)
# print(obi)
es = EarlyStopping(monitor="loss", mode="auto")

# for _ in range(10):
#     for qn in attempts_by_q:
#         print("training {}, pre est beta = {}".format(qn.id, qn.pred_beta.get_weights()[0][0]))
#         thetas = array([ tup[0]["pred_theta"].get_weights()[0][0] for tup in attempts_by_q[qn]])
#         passed = array([ tup[2] for tup in attempts_by_q[qn] ])     
#         obi = numpy.ones(len(thetas))
#         qn.model.fit([thetas, obi],passed, batch_size=1, verbose=1, epochs=100, callbacks=[es])
#         print("{}, post est beta = {}. real {}".format(qn.id, qn.pred_beta.get_weights()[0][0], qn.beta))

#     for student_obj in psi_list:
#         student = student_obj["name"]
#         print("training {}".format(student))
#         print("Pre est {} has theta: {}".format(student, student_obj["pred_theta"].get_weights()[0][0]))
#         betas = array([ tup[1].pred_beta.get_weights()[0][0] for tup in attempts_by_psi[student] ])
#         passed = array([ tup[2] for tup in attempts_by_psi[student] ]).reshape(-1,1)
#         print(betas.shape)
#         print(passed.shape)
#         obi = numpy.ones(len(betas))
#         student_obj["model"].fit([obi, betas],passed, batch_size=1, verbose=1, epochs=100, callbacks=[es])
#         print("Post est {} has theta: {}, real {}".format(student, student_obj["pred_theta"].get_weights()[0][0], student_obj["theta"]))
    
random.seed(666)
calibration_mode = "NEURAL"
repz = (10 if calibration_mode=="NEURAL" else 1)
es = EarlyStopping(monitor="loss", mode="auto", min_delta=0.001)
student_att_ct = Counter()
pat_ct = 10
b_del_tol = 0.1
th_del_tol = 0.1
last_th_rmse = 1000
last_b_rmse = 1000
freeze_betas = False
freeze_thetas = False
for _ in range(100):
    if calibration_mode=="NEURAL":
        shuffle(master_qs)
        shuffle(psi_list)
        for q in master_qs:
            print(q.id)
            b_del = b_del_tol+1
            thetas = [ tup[0]["pred_theta"].get_weights()[0][0] for tup in attempts_by_q[q]]
#             print("got  thetas")
            pfs = [ tup[2] for tup in attempts_by_q[q]]
#             print("got pfs")
            preb = q.pred_beta.get_weights()[0][0]
#             while(b_del > tol):
#             print("fit q")
            if not freeze_betas:
                q.model.fit([array(thetas), numpy.ones(len(thetas))],[array(pfs)], verbose=0, batch_size=1, epochs=100, callbacks=[es])
            postb = q.pred_beta.get_weights()[0][0]      
#                 b_del = abs(postb - preb)
#                 print("bdel",b_del)
            print("q{} true({}), preb {} postb {}".format(q.id, q.beta, preb, postb))

            if not freeze_thetas:
                psi_sublist = [tup[0] for tup in attempts_by_q[q]]
                for _student in psi_sublist:
                    th_del = th_del_tol+1
                    betas = [ tup[1].pred_beta.get_weights()[0][0] for tup in attempts_by_psi[_student["name"]]]
                    pfs = [ tup[2] for tup in attempts_by_psi[_student["name"]]]
                    preth= _student["pred_theta"].get_weights()[0][0] 
            #             while(th_del > tol):
            #             print("fit psi")
                    _student["model"].fit([numpy.ones(len(betas)), array(betas)],[array(pfs)], verbose=0, batch_size=1, epochs=100, callbacks=[es])
                    postth= _student["pred_theta"].get_weights()[0][0]
            #                 th_del = abs(postth - preth)
            #                 print("th_Del",th_del)
                    print("s{}: true({}), {} -> {}".format(_student["name"], _student["theta"], preth, postth))

    else:
        for q in master_qs:
            new_beta = calc_better_beta(q.pred_beta_sc, attempts_by_q[q])#[0:student_att_ct[student["name"]]])
            q.pred_beta_sc = new_beta
        for student in psi_list:
            theta = student["pred_theta_sc"]
            new_theta = calc_better_theta(theta, attempts_by_psi[student["name"]])
            student["pred_theta_sc"] = new_theta
            
            
    if calibration_mode=="NEURAL":
        th_rmse = mean([(s["theta"] - s["pred_theta"].get_weights()[0][0])**2 for s in psi_list])
        b_rmse = mean([(q.beta - q.pred_beta.get_weights()[0][0])**2 for q in master_qs])
    else:
        th_rmse = mean([(s["theta"] - s["pred_theta_sc"])**2 for s in psi_list])
        b_rmse = mean([(q.beta - q.pred_beta_sc)**2 for q in master_qs])

    if (last_b_rmse - b_rmse) > b_del_tol:
        last_b_rmse = b_rmse
        print("b mse:", b_rmse, "del:", (last_b_rmse - b_rmse))
    else:
        freeze_betas = True

    if (last_th_rmse - th_rmse) > th_del_tol:
        last_th_rmse = th_rmse
        input("th mse: {} del: {}".format(th_rmse,(last_th_rmse - th_rmse)))
    else:
        freeze_thetas = True
 
    if freeze_betas and freeze_thetas: # NO MORE CHANGE POSSIBLE!
        break

print("final beta rmse = {}".format(b_rmse))
print("final theta rmse = {}".format(th_rmse))
        

5
q5 true(0), preb [ 0.57880729] postb [ 0.50049323]
sPHUTHY CORE : true(0), [ 1.19720256] -> [ 1.28161216]
sDYG WOLA : true(2), [ 1.63126945] -> [ 1.86485875]
sCYT BAG : true(2), [ 1.59227765] -> [ 1.94220185]
sHAPHU DOKO : true(1), [ 3.18691468] -> [ 3.14212561]
sTHOS TOPHE : true(2), [ 2.77069402] -> [ 2.9724369]
sNUG KIHOTH : true(1), [ 1.79902899] -> [ 1.78087866]
sVIBOCH PHIK : true(0), [ 0.02506085] -> [ 0.10928648]
sKOK RACHE : true(0), [ 0.55760384] -> [ 0.52255678]
sMYM WYPHUTH : true(1), [ 1.07830691] -> [ 1.26782537]
sCHESI SIWI : true(3), [ 2.69377875] -> [ 2.81053758]
sNIBIR PHIFAS : true(3), [ 2.29656386] -> [ 2.27903247]
sBIH NAS : true(2), [ 2.03831887] -> [ 2.46238422]
sSOD GYGI : true(2), [ 1.86216187] -> [ 1.74630618]
sJAVIP THAFY : true(3), [ 2.75956774] -> [ 2.88112259]
sRYT VYHA : true(1), [ 1.15274048] -> [ 1.12337923]
sJYYIY PHAPHYT : true(2), [ 2.64360094] -> [ 2.86653852]
sHUCHAPH VEPH : true(2), [ 2.51864457] -> [ 2.83508182]
sMEG CHISY : true(1), [ 1.627862

sLOCHYF JACHAY : true(0), [ 1.36152589] -> [ 1.43214154]
sDAL SUN : true(2), [ 1.86452723] -> [ 1.82444799]
sTHIRE COTH : true(4), [ 3.86886191] -> [ 3.97542667]
sHUB BOHIJ : true(0), [ 0.] -> [ 0.]
sHEP KEC : true(3), [ 3.91235995] -> [ 3.88185024]
sCAWED CUTO : true(0), [ 0.09285202] -> [ 0.04964592]
sWUM LILI : true(4), [ 3.79893112] -> [ 3.90689397]
sGAKYR YOLUY : true(1), [ 0.15352833] -> [ 0.12960699]
sMURI DETIV : true(0), [ 0.03308672] -> [ 0.02099178]
sFYPH WECHUV : true(0), [ 0.02486583] -> [ 0.03564024]
sBYC YAW : true(1), [ 1.38900352] -> [ 1.37348819]
sFAL YEH : true(2), [ 2.8062892] -> [ 2.7585659]
sYET MOYIR : true(3), [ 3.97623396] -> [ 3.9100306]
sTISOC LEKEY : true(1), [ 1.24148726] -> [ 1.09956539]
sYIK SUPHI : true(2), [ 1.76834333] -> [ 1.93009365]
sGICHOH CHOVAH : true(1), [ 3.38820314] -> [ 3.2832582]
sMUMI THOFEC : true(2), [ 0.84118587] -> [ 0.66408932]
sFIGI PYTU : true(2), [ 2.36636353] -> [ 2.16134024]
sDAGIS DUT : true(0), [ 0.] -> [ 0.]
sMIMI NOPY : true(1

sNAM PHYVU : true(3), [ 3.95175886] -> [ 3.74438262]
sCATH PIJAT : true(4), [ 4.] -> [ 4.]
sRYC JYLEN : true(4), [ 3.94821239] -> [ 3.90800667]
sYEPH GYNYP : true(1), [ 1.91823518] -> [ 1.71168017]
sYAVA MAY : true(3), [ 2.37067938] -> [ 2.40190101]
sRIWUH CHEH : true(1), [ 1.42874455] -> [ 1.13747478]
sCHOV POP : true(3), [ 3.82747936] -> [ 3.70150828]
sJIP NOMO : true(1), [ 0.78974605] -> [ 0.54027867]
sCEVUL JYTH : true(0), [ 1.27434385] -> [ 1.17357993]
sSIMOF BADU : true(3), [ 3.33506274] -> [ 3.06773496]
sCUKER PEPHU : true(1), [ 1.24307513] -> [ 1.09430063]
sSON SUJUP : true(0), [ 0.04100347] -> [ 0.07021389]
sPABEL FUP : true(0), [ 0.2488886] -> [ 0.03007465]
sPYKY GIRU : true(2), [ 2.35909891] -> [ 2.34586573]
sCHUDU RODYV : true(2), [ 2.14477134] -> [ 2.33306408]
sPAWIL TAVO : true(2), [ 2.39779925] -> [ 2.20735669]
sCHET LINAP : true(1), [ 0.09821643] -> [ 0.03091781]
sPHATH YOPY : true(0), [ 0.] -> [ 0.]
sPHATYT THIV : true(0), [ 1.21966946] -> [ 1.32268691]
sTHAP WUN : tru

sPIKIP PHYDOJ : true(2), [ 2.8043766] -> [ 2.57193851]
sSIN YEPHI : true(2), [ 3.45126534] -> [ 3.31493306]
sPOCOG THEM : true(1), [ 0.55697781] -> [ 0.66650873]
sRUP JANYT : true(4), [ 3.80420947] -> [ 3.89577556]
sRIHUJ KUR : true(2), [ 2.2320807] -> [ 2.30005717]
sNEJO TIBY : true(2), [ 1.78997123] -> [ 1.59730899]
sPYWATH ROTH : true(0), [ 0.] -> [ 0.]
sDIH SIVE : true(3), [ 2.9890511] -> [ 2.53422689]
sPHYHIS RYR : true(1), [ 1.27324665] -> [ 1.18469036]
sPHIRAY WIPIPH : true(3), [ 3.06986618] -> [ 3.35975981]
sCULA GIHAM : true(2), [ 1.61113584] -> [ 1.8421458]
sKUR PIVA : true(0), [ 0.] -> [ 0.]
sHOFUC WUCH : true(1), [ 0.49482706] -> [ 0.39305672]
6
q6 true(4), preb [ 3.80384207] postb [ 3.319417]
sPHUTHY CORE : true(0), [ 1.13524699] -> [ 1.13903558]
sDYG WOLA : true(2), [ 1.74447656] -> [ 1.68934953]
sCYT BAG : true(2), [ 1.79246187] -> [ 1.74540317]
sHAPHU DOKO : true(1), [ 3.3818121] -> [ 3.07625127]
sTHOS TOPHE : true(2), [ 2.69842815] -> [ 2.90670681]
sNUG KIHOTH : true(1

sTYHA KEG : true(0), [ 0.50423741] -> [ 0.58607227]
sTUW SETIL : true(3), [ 1.66331637] -> [ 1.58159721]
sTOCHAH MYTI : true(4), [ 2.61734343] -> [ 2.5728333]
sWYTHE VYKO : true(1), [ 1.85052943] -> [ 1.71856332]
sBIM GYKOS : true(0), [ 0.66779172] -> [ 0.50693798]
sSECHUY VUPH : true(0), [ 0.57420576] -> [ 0.40032005]
sSENOD PYH : true(0), [ 1.03064871] -> [ 1.19198883]
sNUM HIGO : true(0), [ 0.04607927] -> [ 0.03033704]
sFICHUG GICHOPH : true(3), [ 2.78292704] -> [ 2.6062119]
sHYR WAG : true(1), [ 0.95765603] -> [ 1.17780137]
sHYVI FYKO : true(2), [ 2.59928489] -> [ 2.82222342]
sTUV KUTHON : true(4), [ 4.] -> [ 4.]
sFYV WYBE : true(0), [ 0.] -> [ 0.00754505]
sLOCHYF JACHAY : true(0), [ 1.19488668] -> [ 1.25884771]
sDAL SUN : true(2), [ 1.64452076] -> [ 1.76531208]
sTHIRE COTH : true(4), [ 3.73151112] -> [ 3.64658809]
sHUB BOHIJ : true(0), [ 0.] -> [ 0.]
sHEP KEC : true(3), [ 3.65452099] -> [ 3.77416229]
sCAWED CUTO : true(0), [ 0.] -> [ 0.04948236]
sWUM LILI : true(4), [ 3.67218399] 

...

sRYC JYLEN : true(4), [ 3.71501207] -> [ 3.62636447]
sYEPH GYNYP : true(1), [ 1.57450461] -> [ 1.46968627]
sYAVA MAY : true(3), [ 2.10368252] -> [ 2.31300473]
sRIWUH CHEH : true(1), [ 1.066149] -> [ 1.17667496]
sCHOV POP : true(3), [ 3.63527942] -> [ 3.45982218]
sJIP NOMO : true(1), [ 0.40011185] -> [ 0.36393794]
sCEVUL JYTH : true(0), [ 1.06433606] -> [ 1.03766]
sSIMOF BADU : true(3), [ 3.11255121] -> [ 3.1767807]
sCUKER PEPHU : true(1), [ 1.1232239] -> [ 1.10914695]
sSON SUJUP : true(0), [ 0.] -> [ 0.]
sPABEL FUP : true(0), [ 0.04723765] -> [ 0.03275721]
sPYKY GIRU : true(2), [ 1.98220336] -> [ 2.02931476]
sCHUDU RODYV : true(2), [ 2.17415714] -> [ 2.26421237]
sPAWIL TAVO : true(2), [ 1.98041272] -> [ 2.02604651]
sCHET LINAP : true(1), [ 0.04773448] -> [ 0.]
sPHATH YOPY : true(0), [ 0.] -> [ 0.]
sPHATYT THIV : true(0), [ 1.10671008] -> [ 1.17698216]
sTHAP WUN : true(4), [ 3.98869896] -> [ 3.97577453]
sPYCHO LYL : true(3), [ 3.00322509] -> [ 3.18335176]
sNEDAN HOWY : true(1), [ 0.3596

sPIKIP PHYDOJ : true(2), [ 2.71392512] -> [ 2.53653336]
sSIN YEPHI : true(2), [ 3.21849298] -> [ 3.1190393]
sPOCOG THEM : true(1), [ 0.51092005] -> [ 0.46831787]
sRUP JANYT : true(4), [ 3.70242882] -> [ 3.49770904]
sRIHUJ KUR : true(2), [ 2.08813477] -> [ 2.21198416]
sNEJO TIBY : true(2), [ 1.68602681] -> [ 1.63314033]
sPYWATH ROTH : true(0), [ 0.] -> [ 0.]
sDIH SIVE : true(3), [ 2.74684978] -> [ 2.6457653]
sPHYHIS RYR : true(1), [ 1.09372818] -> [ 1.08608878]
sPHIRAY WIPIPH : true(3), [ 3.10985613] -> [ 3.03609014]
sCULA GIHAM : true(2), [ 1.6718477] -> [ 1.58519697]
sKUR PIVA : true(0), [ 0.] -> [ 0.]
sHOFUC WUCH : true(1), [ 0.43353388] -> [ 0.32745349]
5
q5 true(0), preb [ 0.50049323] postb [ 0.26952168]
sPHUTHY CORE : true(0), [ 1.04127419] -> [ 1.04049015]
sDYG WOLA : true(2), [ 1.63976312] -> [ 1.6068995]
sCYT BAG : true(2), [ 1.59049046] -> [ 1.62404382]
sHAPHU DOKO : true(1), [ 3.18866587] -> [ 3.02749062]
sTHOS TOPHE : true(2), [ 2.54843783] -> [ 2.62682128]
sNUG KIHOTH : tru

sNEDAN HOWY : true(1), [ 0.33652902] -> [ 0.45804062]
sYYNY NID : true(2), [ 1.65152454] -> [ 1.72050083]
sGEGE FUTY : true(1), [ 0.39098582] -> [ 0.47841135]
sLUR DYFU : true(3), [ 2.46127772] -> [ 2.59269929]
sTYHA KEG : true(0), [ 0.36325738] -> [ 0.60412645]
sTUW SETIL : true(3), [ 1.5336746] -> [ 1.68508542]
sTOCHAH MYTI : true(4), [ 2.69066477] -> [ 2.66060305]
sWYTHE VYKO : true(1), [ 1.49628961] -> [ 1.55508971]
sBIM GYKOS : true(0), [ 0.2887907] -> [ 0.56626564]
sSECHUY VUPH : true(0), [ 0.37354046] -> [ 0.31048745]
sSENOD PYH : true(0), [ 0.99915695] -> [ 1.0570035]
sNUM HIGO : true(0), [ 0.] -> [ 0.]
sFICHUG GICHOPH : true(3), [ 2.60053921] -> [ 2.75867963]
sHYR WAG : true(1), [ 1.05319905] -> [ 1.09457946]
sHYVI FYKO : true(2), [ 2.51207995] -> [ 2.55501175]
sTUV KUTHON : true(4), [ 4.] -> [ 4.]
sFYV WYBE : true(0), [ 0.] -> [ 0.00195218]
sLOCHYF JACHAY : true(0), [ 1.11546397] -> [ 0.94663709]
sDAL SUN : true(2), [ 1.70246434] -> [ 1.47163939]
sTHIRE COTH : true(4), [ 3.59

sNUG KIHOTH : true(1), [ 1.49397361] -> [ 1.68892455]
sVIBOCH PHIK : true(0), [ 0.] -> [ 0.]
sKOK RACHE : true(0), [ 0.56035924] -> [ 0.3895503]
sMYM WYPHUTH : true(1), [ 1.12572384] -> [ 0.92620474]
sCHESI SIWI : true(3), [ 2.6498754] -> [ 2.5394609]
sNIBIR PHIFAS : true(3), [ 2.27521825] -> [ 2.09122992]
sBIH NAS : true(2), [ 2.00783277] -> [ 2.03681254]
sSOD GYGI : true(2), [ 1.63146818] -> [ 1.42655253]
sJAVIP THAFY : true(3), [ 2.53771496] -> [ 2.56152248]
sRYT VYHA : true(1), [ 0.99320656] -> [ 1.01892984]
sJYYIY PHAPHYT : true(2), [ 2.51484847] -> [ 2.65273476]
sHUCHAPH VEPH : true(2), [ 2.68092537] -> [ 2.62183332]
sMEG CHISY : true(1), [ 1.5656029] -> [ 1.62042129]
sKUL PYY : true(4), [ 3.99167085] -> [ 4.]
sBAG CHYPHET : true(0), [ 0.] -> [ 0.]
sHOTUTH PHECOS : true(2), [ 1.05273104] -> [ 1.11588693]
sMODIC NAWACH : true(3), [ 2.59047627] -> [ 2.60943532]
sNAD PUH : true(4), [ 4.] -> [ 4.]
sDYLO JYPH : true(0), [ 1.13645983] -> [ 1.12117934]
sTHEMYS POV : true(0), [ 0.4165996

sDAL SUN : true(2), [ 1.58703887] -> [ 1.67005277]
sTHIRE COTH : true(4), [ 3.43894339] -> [ 3.50794911]
sHUB BOHIJ : true(0), [ 0.] -> [ 0.]
sHEP KEC : true(3), [ 3.49551058] -> [ 3.68825912]
sCAWED CUTO : true(0), [ 0.] -> [ 0.]
sWUM LILI : true(4), [ 3.65246081] -> [ 3.57009435]
sGAKYR YOLUY : true(1), [ 0.] -> [ 0.00618273]
sMURI DETIV : true(0), [ 0.00650134] -> [ 0.]
sFYPH WECHUV : true(0), [ 0.] -> [ 0.]
sBYC YAW : true(1), [ 0.96086347] -> [ 1.16145277]
sFAL YEH : true(2), [ 2.61051631] -> [ 2.58552003]
sYET MOYIR : true(3), [ 3.46730113] -> [ 3.60657287]
sTISOC LEKEY : true(1), [ 1.19371724] -> [ 1.06666005]
sYIK SUPHI : true(2), [ 1.61773634] -> [ 1.41691601]
sGICHOH CHOVAH : true(1), [ 3.02466321] -> [ 2.9181459]
sMUMI THOFEC : true(2), [ 0.4279829] -> [ 0.39147142]
sFIGI PYTU : true(2), [ 2.12416482] -> [ 2.02610636]
sDAGIS DUT : true(0), [ 0.] -> [ 0.]
sMIMI NOPY : true(1), [ 0.98140591] -> [ 1.06717753]
sTENI BIHO : true(1), [ 0.] -> [ 0.]
sPHATY HOK : true(2), [ 2.081796

In [12]:
mse_ths = []
for psi in psi_list:
    real_th = psi["theta"]
    if calibration_mode=="NEURAL":
        pred_th = psi["pred_theta"].get_weights()[0][0]
    else:
        pred_th = psi["pred_theta_sc"]        
    mse_ths.append( (real_th - pred_th)**2 )
    print(psi["name"],":",real_th,"vs",pred_th)
print("MSE in theta: {}".format(mean(mse_ths)))

mse_bs = []
for q in master_qs:
    real_b = q.beta
    if calibration_mode=="NEURAL":
        pred_b = q.pred_beta.get_weights()[0][0]
    else:
        pred_b = q.pred_beta_sc
    mse_bs.append( (real_b - pred_b)**2 )
    print(real_b, "vs", pred_b)
print("MSE in beta: {}".format(mean(mse_bs)))


RYB NACAT  : 0 vs [ 0.06975579]
NEJO TIBY  : 2 vs [ 3.33991313]
NIT MIJUS  : 2 vs [ 1.66868472]
DAGIS DUT  : 0 vs [ 0.05132208]
NATA CHEPH  : 3 vs [ 3.41513324]
BUW JEKUPH  : 1 vs [ 0.]
CAWED CUTO  : 0 vs [ 0.60339421]
BAREP RYWYV  : 3 vs [ 2.43889213]
NAM PHYVU  : 3 vs [ 3.55235863]
TETHEK YIF  : 1 vs [ 0.79910707]
NUVI HARYPH  : 0 vs [ 0.]
JUHUM PEKU  : 2 vs [ 2.81118441]
WYTHE VYKO  : 1 vs [ 0.17112941]
HYSET GOPHA  : 0 vs [ 0.76190972]
BYC YAW  : 1 vs [ 1.35805225]
HEF SUL  : 2 vs [ 1.34012556]
NOM VOF  : 2 vs [ 3.3900094]
YUC NAPO  : 3 vs [ 2.87795687]
TUC BOD  : 1 vs [ 0.85264307]
GIHYN KAW  : 2 vs [ 1.10871601]
VYF YUKAG  : 0 vs [ 0.]
HUDO DOVY  : 3 vs [ 2.25604916]
LYBY CHYT  : 3 vs [ 3.71749473]
KEJUK DUKY  : 0 vs [ 0.]
HYTHOP JYJAG  : 0 vs [ 0.]
FAME MIG  : 4 vs [ 3.53463125]
CHIF MEPHY  : 4 vs [ 4.]
SUTITH SUNOCH  : 3 vs [ 2.97205758]
CAJ FIJ  : 3 vs [ 3.47198749]
BAS TUJ  : 1 vs [ 1.32178617]
HILYM WAKA  : 4 vs [ 4.]
FYLA JYY  : 4 vs [ 4.]
HEP KEC  : 3 vs [ 2.96802926]
DUTY

PHATH YOPY  : 0 vs [ 1.30125153]
JYCH VOSOM  : 3 vs [ 2.94893503]
SYSO SEJ  : 4 vs [ 4.]
WIS NEHU  : 1 vs [ 0.75092113]
FAS NEVE  : 2 vs [ 0.78981304]
FAPY HOTUY  : 4 vs [ 4.]
DIP DATHI  : 2 vs [ 1.75780749]
FAL YEH  : 2 vs [ 1.74686575]
NUM HIGO  : 0 vs [ 0.]
HASE HEFE  : 1 vs [ 1.2811203]
HICH HIWE  : 1 vs [ 1.30364132]
LEYE YOPHAB  : 3 vs [ 3.81748533]
RORU LACHO  : 0 vs [ 0.66987669]
WEPHOL BUWIV  : 3 vs [ 3.51101351]
THUD DECIW  : 0 vs [ 1.21140349]
RIYIV WOPH  : 4 vs [ 4.]
CHOGUC YORY  : 3 vs [ 2.41205096]
MYV PHOCH  : 3 vs [ 3.26165342]
CYT NAPH  : 3 vs [ 2.80127716]
BEME HABOPH  : 3 vs [ 2.91989493]
JEJ WED  : 2 vs [ 1.40553367]
LOCHYF JACHAY  : 0 vs [ 0.59451759]
SOPH YAV  : 2 vs [ 1.85632527]
THECHY YAVATH  : 4 vs [ 3.94500899]
GECH GOL  : 0 vs [ 0.01695997]
PAGI NODYTH  : 3 vs [ 2.80154085]
PUR TUPI  : 3 vs [ 2.28795409]
TOHIM JOT  : 0 vs [ 0.08016153]
JASAG WIY  : 1 vs [ 1.18254972]
LIPOCH PHODE  : 0 vs [ 0.]
TOPIB THOTI  : 4 vs [ 3.88624287]
SIMOF BADU  : 3 vs [ 2.51141477

PACH LUBEJ  : 1 vs [ 0.]
PHYMO MACO  : 3 vs [ 1.77530897]
RYWOB TEF  : 4 vs [ 3.69354725]
RAYO CHES  : 1 vs [ 1.20973623]
THINOG BUKU  : 1 vs [ 2.05178881]
CHYHEK WAV  : 1 vs [ 1.29476893]
CAPE LICHYL  : 4 vs [ 4.]
MAGUS FATHAS  : 3 vs [ 2.76659441]
PABEL FUP  : 0 vs [ 0.]
WEGATH DOJE  : 3 vs [ 2.90789342]
BOH MYTYG  : 4 vs [ 4.]
NENI WEVIY  : 1 vs [ 1.76130211]
NODU CHIG  : 2 vs [ 1.30628157]
FES CHUTE  : 3 vs [ 4.]
TEBAL FUFI  : 3 vs [ 3.35610008]
LEYIB PHOF  : 0 vs [ 1.41089964]
FIGO KOJO  : 4 vs [ 4.]
JESA KEPA  : 4 vs [ 4.]
CEVUL JYTH  : 0 vs [ 0.08029846]
COCHA YOJOL  : 0 vs [ 0.01565597]
TAWITH GISEM  : 1 vs [ 0.68772727]
BOTES YEYEL  : 3 vs [ 3.15090895]
WELI JARYM  : 2 vs [ 2.74149823]
TIM CAPA  : 0 vs [ 0.]
CIK DUPE  : 1 vs [ 1.18790615]
THIRY GUKI  : 3 vs [ 3.84406972]
PUPHUJ SOCOPH  : 0 vs [ 0.1024247]
PHOFE TOY  : 3 vs [ 3.27131152]
SEDIH FAJ  : 0 vs [ 0.0282806]
LEPHO RIKI  : 4 vs [ 3.71338153]
FEV RYH  : 1 vs [ 2.32988501]
SYFUG HIHI  : 4 vs [ 4.]
JUPHA CEKAH  : 2 vs [ 1